In [1]:
from miditok import REMI, get_midi_programs
from miditoolkit import MidiFile

In [2]:
pitch_range = range(21, 109)
beat_res = {(0, 4): 8, (4, 12): 4}
nb_velocities = 32

In [25]:
tokenizer = REMI()
midi = MidiFile("Piano_collections_all/Train\\appass_3.mid")

In [26]:
tokens = tokenizer.midi_to_tokens(midi)

In [5]:
len(tokenizer.vocab)

218

In [27]:
print(len(tokens))
print(len(tokens[0]))
#print(len(tokens[1]))


2
14448


In [33]:
tokens[1][-10:]

[145, 44, 110, 145, 49, 110, 145, 52, 114, 145]

In [34]:
a = [1,2,3,4,5,86,6,1,5,6,7,8]

In [8]:
# extract events from current event until next "position" event
def extract_position(position,array):
    extracted_array = [array[position]]
    position += 1
    done = False
    while array[position] < 186 and array[position] != 1: # when not position event
        extracted_array.append(array[position])
        position += 1
        if position == len(array):
            done = True
            break
    return position, extracted_array, done

In [9]:
# tract events from current event until next "bar" event
def extract_bar(position,array):
    extracted_array = [array[position]]
    position += 1
    done = False
    while array[position] != 1: # when not position event
        extracted_array.append(array[position])
        position += 1
        if position == len(array):
            done = True
            break
    return position, extracted_array, done # current position pointing at bar event

In [11]:
# 0 - padding
# 1 - bar
# 2 - 89 - pitch
# 90 - 121 - velocity
# 122 - 185 - duration
# 187 - 217 - position


def combine_encoding(left,right):
    position1 = 1 # left
    position2 = 1 # right
    combined_events= [1]
    left_finish_first = False
    
    while True:
        # if left event is bar
        if left[position1] == 1:
            # if right event is not bar
            if right[position2] != 1:
                # right hand much reach next bar to syncronise the track
                # extract everything of right events until next bar
                position2, temparray,done = extract_bar(position2,right)
                
                # append temparray to combined_events
                combined_events.extend(temparray)
                
                # if right hand reached the end of its track
                if done:
                    left_finish_first = False
                    break
                
            # we have two bar events, append bar event into 
            #  combined_events, and move position by 1
            combined_events.append(1)
            position1 += 1
            position2 += 1
            continue
            
        # if right event is bar
        if right[position2] == 1:
            # if left event is not bar
            if left[position1] != 1:
                # left hand must reach next bar to syncronise the track
                # extract everything of left events until next bar
                position1, temparray,done = extract_bar(position1,left)
                
                # append temparray to combined_events
                combined_events.extend(temparray)
                
                # if left hand reached the end of its track
                if done:
                    left_finish_first = True
                    break
                
            # we have two bar events, append bar event into 
            #  combined_events, and move position by 1
            combined_events.append(1)
            position1 += 1
            position2 += 1
            continue
        
        left_position_event = left[position1]
        right_position_event = right[position2]
        
        # if left position event is smaller than right position event
        #   move left position until the next position event or bar event
        if left_position_event < right_position_event:
            position1, temparray,done = extract_position(position1,left)
            combined_events.extend(temparray)
            # if left hand reached the end of its track
            if done:
                left_finish_first = True
                break
        elif left_position_event > right_position_event:
            # move right events until next position event or bar event
            position2, temparray,done = extract_position(position2,right)
            combined_events.extend(temparray)
            # if right hand reached the end of its track
            if done:
                left_finish_first = False
                break
        else:
            # if two event position are the same
            #  move both events and append them all
            position1, temparray,done1 = extract_position(position1,left)
            position2, temparray2, done2 = extract_position(position2,right)
            temparray.extend(temparray2[1:])
            combined_events.extend(temparray)
            
            if done1:
                left_finish_first = True
                break
            if done2:
                left_finish_first = False
                break
    
    # combining the remining track of unfinished track
    if left_finish_first:
        # trackdown remaining right hand track
        combined_events.extend(right[position2:])
    else:
        combined_events.extend(left[position1:])
    
    return combined_events
        
        

    
    

In [56]:
a = [1,186,2,2,187,3,3,1,190,3,3]
b = [1,186,5,5,1,189,6,6]


In [47]:
# get fullnames of training directory for classical music
import os
names = os.listdir("Pop-music")
names[:10]
# input paths
fnames = map(lambda name:os.path.join("Classical Music/Train",name),names)
fnames = list(fnames)
fnames[:10]

['Classical Music/Train\\01pasali.mid',
 'Classical Music/Train\\01promes.mid',
 'Classical Music/Train\\02holnit.mid',
 'Classical Music/Train\\02regven.mid',
 'Classical Music/Train\\03manger.mid',
 'Classical Music/Train\\04adeste.mid',
 'Classical Music/Train\\04gita.mid',
 'Classical Music/Train\\05littre.mid',
 'Classical Music/Train\\06chimes.mid',
 'Classical Music/Train\\06dealpi.mid']

In [17]:
# extract only the name from path
os.path.basename(os.path.normpath(fnames[0]))

'01pasali.mid'

In [39]:
# apply combining track algorithm to each of them based on number of tracks it has
import numpy as np

tokenizer = REMI() # default parameters

for path in fnames:
    # prepare the midi file
    midi = MidiFile(path)
    # tokenize the midi file
    try: tokens = tokenizer.midi_to_tokens(midi)
    except: continue
    # check number of tracks, usually two, this determines how many conbines to be done
    n_track = len(tokens)
    
    # define output path
    name = os.path.basename(os.path.normpath(path)) # extract name of the song
    output_path = os.path.join("Classical Music/Val",name)
    
    # if only one track
    if n_track == 1:
        tokensMidi = tokenizer.tokens_to_midi([tokens[0]],
                                                 get_midi_programs(midi))
        tokensMidi.dump(output_path)
        continue
    
    # combine the first two tracks
    combined_track = combine_encoding(tokens[0],tokens[1])
    
    # combine the rest tracks if there is more
    for i in np.arange(2,n_track):
        combined_track = combine_encoding(tokens[i],combined_track)
    
    # write out to output directory
    
    #  convert tokenization to midi
    combined_trackMidi = tokenizer.tokens_to_midi([combined_track],
                                                 get_midi_programs(midi))
    combined_trackMidi.dump(output_path)
    
    

Classical Music/Val\03linvit.mid
Classical Music/Val\05rimpro.mid
Classical Music/Val\07parten.mid
Classical Music/Val\09evnbel.mid
Classical Music/Val\6_variat.mid
Classical Music/Val\abdul.mid
Classical Music/Val\aj_lis13.mid
Classical Music/Val\allegret.mid
Classical Music/Val\allegro_.mid
Classical Music/Val\alp-sea.mid
Classical Music/Val\baga05.mid
Classical Music/Val\ballade3.mid
Classical Music/Val\be-ps-04.mid
Classical Music/Val\be-pv13r.mid
Classical Music/Val\beet1mv2.mid
Classical Music/Val\beethoven_hammerklavier_3.mid
Classical Music/Val\beethoven_opus22_4.mid
Classical Music/Val\beet_3_2.mid
Classical Music/Val\benedict.mid
Classical Music/Val\chasse.mid
Classical Music/Val\chonoc11.mid
Classical Music/Val\chopin_e.mid
Classical Music/Val\chopol10.mid
Classical Music/Val\choval02.mid
Classical Music/Val\choval13.mid
Classical Music/Val\chpn-p22.mid
Classical Music/Val\chpn-p4.mid
Classical Music/Val\chpn_op27_1.mid
Classical Music/Val\chpn_op53.mid
Classical Music/Val\c

In [45]:
# make sure now I only have one track - val
for path in fnames:
    # prepare the midi file
    midi = MidiFile(path)
    # tokenize the midi file
    tokens = tokenizer.midi_to_tokens(midi)
    # check number of tracks, usually two, this determines how many conbines to be done
    n_track = len(tokens)
    if n_track != 1:
        print(path)
        print(n_track)
print("all done!")

all done!


In [48]:
# make sure now I only have one track - train
for path in fnames:
    # prepare the midi file
    midi = MidiFile(path)
    # tokenize the midi file
    tokens = tokenizer.midi_to_tokens(midi)
    # check number of tracks, usually two, this determines how many conbines to be done
    n_track = len(tokens)
    if n_track != 1:
        print(path)
        print(n_track)
print("all done!")

all done!
